# Make the plot for MI and deconvolution relationship, 01/29/2024

In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
from jax import config
config.update("jax_enable_x64", True)
import numpy as np

import sys
sys.path.insert(0, '/home/lkabuli_waller/workspace/EncodingInformation/')
sys.path.insert(0, '/home/lkabuli_waller/workspace/EncodingInformation/imager_experiments')
from leyla_fns import *
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
print(os.environ.get('PYTHONPATH'))
from cleanplots import * 

In [ ]:
seed_value = 10

# set photon properties 
bias = 10 # in photons
mean_photon_count_list = [20, 40, 60, 80, 100, 150, 160, 200, 250, 300, 320]
max_photon_count = mean_photon_count_list[-1]

# set eligible psfs

psf_names = ['one', 'four', 'diffuser'] # later make it all of them, but haven't gotten diffuser and aperture yet

# MI estimator parameters 
patch_size = 32
num_patches = 10000
bs = 500

## Load MI data and make plots of it
Using updated MI data from 01/17/2024 which is run for the uniform data

The plot has essentially invisible error bars. No more outlier issues

In [ ]:
from cleanplots import *
get_color_cycle()[0]

In [ ]:
mi_folder = ''

### Minimum plot with no error bars

In [ ]:
gaussian_mi_estimates_across_psfs = [] # only keeps the minimum values, no outliers
pixelcnn_mi_estimates_across_psfs = [] # only keeps the minimum values, no outliers

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
for psf_name in psf_names:
    gaussian_across_photons = [] 
    pixelcnn_across_photons = []
    for photon_count in mean_photon_count_list:
        #gaussian_mi_estimate = np.load(mi_folder + 'cifar10_mi_estimates/gaussian_mi_estimate_{}_photon_count_{}_psf.npy'.format(photon_count, psf_name))
        pixelcnn_mi_estimate = np.load(mi_folder + 'cifar10_mi_estimates/pixelcnn_mi_estimate_{}_photon_count_{}_psf.npy'.format(photon_count, psf_name))
        #gaussian_across_photons.append(gaussian_mi_estimate)
        pixelcnn_across_photons.append(pixelcnn_mi_estimate)
        assert pixelcnn_mi_estimate.shape[0] == 4
    #gaussian_mins = np.min(gaussian_across_photons, axis=1)
    pixelcnn_mins = np.min(pixelcnn_across_photons, axis=1)
    ax.plot(mean_photon_count_list, gaussian_mins, '-', label='Gaussian {}'.format(psf_name))
    ax.plot(mean_photon_count_list, pixelcnn_mins, '-', label='PixelCNN {}'.format(psf_name))
    gaussian_mi_estimates_across_psfs.append(gaussian_mins) # only keep mean dataset for use
    pixelcnn_mi_estimates_across_psfs.append(pixelcnn_mins) # only keep mean datas
plt.legend()
plt.title("Gaussian vs. PixelCNN MI Estimates Across Photon Count, CIFAR10, 4 Seeds, Minimums")
plt.ylabel('Estimated Mutual Information')
plt.xlabel('Mean Photon Count')

gaussian_mi_estimates_across_psfs = np.array(gaussian_mi_estimates_across_psfs)
pixelcnn_mi_estimates_across_psfs = np.array(pixelcnn_mi_estimates_across_psfs)

In [ ]:
psf_names_verbose = ['One Lens', 'Two Lens', 'Three Lens', 'Four Lens', 'Five Lens', 'Diffuser', 'Aperture']
plt.figure(figsize=(6, 5))
ax = plt.axes()
for i, modality in enumerate(psf_names_verbose):
        #plt.plot(mean_photon_count_list, gaussian_mi_estimates_across_psfs[i], label = '{} Gaussian'.format(modality), color = get_color_cycle()[i], linestyle='--')
        plt.plot(mean_photon_count_list, pixelcnn_mi_estimates_across_psfs[i], label = '{}'.format(modality), color = get_color_cycle()[i-1]) # manual color correct
plt.legend()
plt.xlabel('Mean Photon Count')
plt.ylabel("Mutual Information (bits per pixel)")
#plt.title('Estimated Mutual Information vs. Mean Photon Count, CIFAR10')
clear_spines(ax)
#plt.savefig('mi_vs_photon_count.pdf', bbox_inches='tight', transparent=True)

### Mean plot with error bars included

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
for psf_name in psf_names:
    gaussian_across_photons = [] 
    pixelcnn_across_photons = []
    for photon_count in mean_photon_count_list:
        #gaussian_mi_estimate = np.load(mi_folder + 'cifar10_mi_estimates/gaussian_mi_estimate_{}_photon_count_{}_psf.npy'.format(photon_count, psf_name))
        pixelcnn_mi_estimate = np.load(mi_folder + 'cifar10_mi_estimates/pixelcnn_mi_estimate_{}_photon_count_{}_psf.npy'.format(photon_count, psf_name))
        if np.max(pixelcnn_mi_estimate) / np.min(pixelcnn_mi_estimate) > 2:
            pixelcnn_mi_estimate[pixelcnn_mi_estimate > 2 * np.min(pixelcnn_mi_estimate)] = np.min(pixelcnn_mi_estimate)
        #gaussian_across_photons.append(gaussian_mi_estimate)
        pixelcnn_across_photons.append(pixelcnn_mi_estimate)
    #error_lo, error_hi, mean = confidence_bars(gaussian_across_photons, 9)
    error_lo_2, error_hi_2, mean_2 = confidence_bars(pixelcnn_across_photons, 11)
    #ax.plot(mean_photon_count_list, mean, '-', label='Gaussian {}'.format(psf_name))
    ax.plot(mean_photon_count_list, mean_2, '-', label='PixelCNN {}'.format(psf_name))
    #ax.fill_between(mean_photon_count_list, error_lo, error_hi, alpha=0.4)
    ax.fill_between(mean_photon_count_list, error_lo_2, error_hi_2, alpha=0.4)
plt.legend()
plt.title("Gaussian vs. PixelCNN MI Estimates Across Photon Count, CIFAR10, 4 Seeds, Means, Outliers Removed")
plt.ylabel('Estimated Mutual Information')
plt.xlabel('Mean Photon Count')

## Load deconvolution data and make plots of it
Use means

In [ ]:
deconvolution_folder = 'unsupervised_wiener_deconvolution_fixed_seed/'

In [ ]:
mse_across_psfs = [] #5 x 9 x 1 array, 5 psfs, 9 photon counts, one value on each 
psnr_across_psfs = [] #5 x 9 x 1 array, 5 psfs, 9 photon counts, one value on each
mse_lists_across_psfs = []
psnr_lists_across_psfs = []
for psf_name in psf_names:
    mse_across_photons = []
    psnr_across_photons = []
    mse_lists_across_photons = []
    psnr_lists_across_photons = []
    for photon_count in mean_photon_count_list:
        mse_list, psnr_list = np.load(deconvolution_folder + 'recon_{}_photon_count_{}_psf.npy'.format(photon_count, psf_name))
        mse_list = np.array(mse_list)
        psnr_list = np.array(psnr_list)
        mean_mse = np.mean(mse_list)
        mean_psnr = np.mean(psnr_list)
        mse_across_photons.append(mean_mse)
        psnr_across_photons.append(mean_psnr)
        mse_lists_across_photons.append(mse_list)
        psnr_lists_across_photons.append(psnr_list)
    mse_across_psfs.append(mse_across_photons)
    psnr_across_psfs.append(psnr_across_photons)
    mse_lists_across_psfs.append(mse_lists_across_photons)
    psnr_lists_across_psfs.append(psnr_lists_across_photons)
mse_across_psfs = np.array(mse_across_psfs)
psnr_across_psfs = np.array(psnr_across_psfs)
mse_lists_across_psfs = np.array(mse_lists_across_psfs)
psnr_lists_across_psfs = np.array(psnr_lists_across_psfs)

In [ ]:
for idx, psf_name in enumerate(psf_names):
    plt.plot(mean_photon_count_list, mse_across_psfs[idx], label='{}'.format(psf_name))
plt.legend()
plt.title("Deconvolution MSE vs. Mean Photon Count, CIFAR10")
plt.ylabel('Mean Squared Error')
plt.xlabel('Mean Photon Count')

## Make figures, include classifier error bars

### Setup

In [ ]:
def marker_for_psf(psf_name):
    if psf_name =='one':
        marker = 'o'
    elif psf_name == 'four':
        marker = 's' 
    elif psf_name == 'diffuser':
        marker = '*'
    elif psf_name == 'uc':
        marker = 'x'
    elif psf_name =='two':
        marker = 'd'
    elif psf_name == 'three':
        marker = 'v'
    elif psf_name == 'five':
        marker = 'p'
    elif psf_name == 'aperture':
        marker = 'P'
    return marker

In [ ]:
# Choose a base colormap
base_colormap = plt.cm.get_cmap('inferno')
# Define the start and end points--used so that high values aren't too light against white background
start, end = 0, 0.88 # making end point 0.8
from matplotlib.colors import LinearSegmentedColormap
# Create a new colormap from the portion of the original colormap
colormap = LinearSegmentedColormap.from_list(
    'trunc({n},{a:.2f},{b:.2f})'.format(n=base_colormap.name, a=start, b=end),
    base_colormap(np.linspace(start, end, 256))
)

min_photons_per_pixel =  min(mean_photon_count_list)
max_photons_per_pixel =  max(mean_photon_count_list)

min_log_photons = np.log(min_photons_per_pixel)
max_log_photons = np.log(max_photons_per_pixel)

def color_for_photon_level(photons_per_pixel):
    log_photons = np.log(photons_per_pixel)
    return colormap((log_photons - min_log_photons) / (max_log_photons - min_log_photons) )

### Update parameters in below block to display the things you want to display, then run the block after to make the figure

In [ ]:
estimator_type = 1 # 0 for gaussian, 1 for pixelcnn
metric_type = 1
valid_psfs = [0, 1, 2]
valid_photon_counts = [20, 40, 80, 150, 160, 300, 320]
print(psf_names)

In [ ]:
confidence_level = 0.9 
# using min-valued MI estimates 
mi_estimate_lists = [gaussian_mi_estimates_across_psfs, pixelcnn_mi_estimates_across_psfs]
deconv_estimate_lists = [mse_across_psfs, psnr_across_psfs]
metric_names = ['MSE', 'PSNR']
# classifier array is classifier_all_trials_across_psfs, 4x9x10 array. 4 psfs, 9 photon counts, 10 trials on each one 

fig, ax = plt.subplots(1, 1, figsize=(7, 5))

mi_list_use = mi_estimate_lists[estimator_type] # use pixelcnn or gaussian, choose pixelcnn 
deconv_list_use = deconv_estimate_lists[metric_type] # use mse or psnr, choose psnr
metric_name = metric_names[metric_type]

for psf_idx, psf_name in enumerate(psf_names):
    if psf_idx in valid_psfs:
        mi_means_across_photons = [] # track mean MI values to make trendline 
        deconv_means_across_photons = [] # track mean MI values to make trendline
        for photon_idx, photon_count in enumerate(mean_photon_count_list):
            if photon_count in valid_photon_counts:
                # load mean values and colors to plot 
                color = color_for_photon_level(photon_count)
                mi_value = mi_list_use[psf_idx][photon_idx] # only use an MI value if the psf is valid, correctly indexed 
                deconv_value = deconv_list_use[psf_idx][photon_idx]
                ax.scatter(mi_value, deconv_value, color=color, marker=marker_for_psf(psf_name), s=50, zorder=100)
                # add to lists to track later 
                mi_means_across_photons.append(mi_value)
                deconv_means_across_photons.append(deconv_value)
        mi_means_across_photons = np.array(mi_means_across_photons)
        deconv_means_across_photons = np.array(deconv_means_across_photons)
        ax.plot(mi_means_across_photons, deconv_means_across_photons, '--', color='grey', alpha=1, linewidth=2)

ax.set_xlabel('Mutual Information (bits per pixel)')
ax.set_ylabel('Deconvolution Performance, {}'.format(metric_name))
clear_spines(ax)


# legend
# ax.scatter([], [], color='k', marker='x', label='No PSF')
ax.scatter([], [], color='k', marker='o', label='One Lens')
# ax.scatter([], [], color='k', marker='d', label='Two Lens')
# ax.scatter([], [], color='k', marker='v', label='Three Lens')
ax.scatter([], [], color='k', marker='s', label='Four Lens')
# ax.scatter([], [], color='k', marker='p', label='Five Lens')
ax.scatter([], [], color='k', marker='*', label='Diffuser')
# ax.scatter([], [], color='k', marker='P', label='Aperture')

ax.legend(loc='lower right', frameon=True)
ax.set_xlim([0, None])



norm = mpl.colors.Normalize(vmin=min_log_photons, vmax=max_log_photons)
sm = plt.cm.ScalarMappable(cmap=colormap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, ticks=(np.log(valid_photon_counts)))
# set tick labels
cbar.ax.set_yticklabels(valid_photon_counts)


cbar.set_label('Mean Photon Count')

In [ ]:
estimator_type = 1 # 0 for gaussian, 1 for pixelcnn
metric_type = 0 # 0 for MSE, 1 for PSNR
valid_psfs = [0, 1, 2, 3, 4, 5, 6]
valid_photon_counts = [20, 40, 80, 150, 160, 300, 320]
print(psf_names)

In [ ]:
confidence_level = 0.9 
# using min-valued MI estimates 
mi_estimate_lists = [gaussian_mi_estimates_across_psfs, pixelcnn_mi_estimates_across_psfs]
deconv_estimate_lists = [mse_across_psfs, psnr_across_psfs]
metric_names = ['MSE', 'PSNR']
# classifier array is classifier_all_trials_across_psfs, 4x9x10 array. 4 psfs, 9 photon counts, 10 trials on each one 

fig, ax = plt.subplots(1, 1, figsize=(7, 5))

mi_list_use = mi_estimate_lists[estimator_type] # use pixelcnn or gaussian, choose pixelcnn 
deconv_list_use = deconv_estimate_lists[metric_type] # use mse or psnr, choose psnr
metric_name = metric_names[metric_type]

for psf_idx, psf_name in enumerate(psf_names):
    if psf_idx in valid_psfs:
        mi_means_across_photons = [] # track mean MI values to make trendline 
        deconv_means_across_photons = [] # track mean MI values to make trendline
        for photon_idx, photon_count in enumerate(mean_photon_count_list):
            if photon_count in valid_photon_counts:
                # load mean values and colors to plot 
                color = color_for_photon_level(photon_count)
                mi_value = mi_list_use[psf_idx][photon_idx] # only use an MI value if the psf is valid, correctly indexed 
                deconv_value = deconv_list_use[psf_idx][photon_idx]
                ax.scatter(mi_value, deconv_value, color=color, marker=marker_for_psf(psf_name), s=50, zorder=100)
                # add to lists to track later 
                mi_means_across_photons.append(mi_value)
                deconv_means_across_photons.append(deconv_value)
        mi_means_across_photons = np.array(mi_means_across_photons)
        deconv_means_across_photons = np.array(deconv_means_across_photons)
        ax.plot(mi_means_across_photons, deconv_means_across_photons, '--', color='grey', alpha=1, linewidth=2)

ax.set_xlabel('Mutual Information (bits per pixel)')
ax.set_ylabel('Deconvolution Performance, {}'.format(metric_name))
clear_spines(ax)


# legend
# ax.scatter([], [], color='k', marker='x', label='No PSF')
ax.scatter([], [], color='k', marker='o', label='One Lens')
ax.scatter([], [], color='k', marker='s', label='Four Lens')
ax.scatter([], [], color='k', marker='*', label='Diffuser')

ax.legend(loc='upper right', frameon=True)
ax.set_xlim([0, None])



norm = mpl.colors.Normalize(vmin=min_log_photons, vmax=max_log_photons)
sm = plt.cm.ScalarMappable(cmap=colormap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, ticks=(np.log(valid_photon_counts)))
# set tick labels
cbar.ax.set_yticklabels(valid_photon_counts)


cbar.set_label('Mean Photon Count')

## Repeat same thing for just one lens, four lens and diffuser, include error bars - final figure

In [ ]:
estimator_type = 1 # 0 for gaussian, 1 for pixelcnn
metric_type = 1
valid_psfs = [0, 1, 2]
valid_photon_counts = [20, 40, 80, 160, 320]
print([psf_names[i] for i in valid_psfs])

In [ ]:
confidence_level = 0.9
# using min-valued MI estimates 
mi_estimate_lists = [gaussian_mi_estimates_across_psfs, pixelcnn_mi_estimates_across_psfs]
#deconv_estimate_lists = [mse_across_psfs, psnr_across_psfs]
deconv_estimate_lists = [mse_lists_across_psfs, psnr_lists_across_psfs] # use full list versions
metric_names = ['MSE', 'PSNR']
# classifier array is classifier_all_trials_across_psfs, 4x9x10 array. 4 psfs, 9 photon counts, 10 trials on each one 

fig, ax = plt.subplots(1, 1, figsize=(7, 5))

mi_list_use = mi_estimate_lists[estimator_type] # use pixelcnn or gaussian, choose pixelcnn 
deconv_list_use = deconv_estimate_lists[metric_type] # use mse or psnr, choose psnr
metric_name = metric_names[metric_type]

for psf_idx, psf_name in enumerate(psf_names):
    if psf_idx in valid_psfs:
        mi_means_across_photons = [] # track mean MI values to make trendline 
        deconv_means_across_photons = [] # track mean deconvolution values to make trendline
        deconv_lower_across_photons = [] # track lower bounds
        deconv_upper_across_photons = [] # track upper bounds
        for photon_idx, photon_count in enumerate(mean_photon_count_list):
            if photon_count in valid_photon_counts:
                # load mean values and colors to plot 
                color = color_for_photon_level(photon_count)
                mi_value = mi_list_use[psf_idx][photon_idx] # only use an MI value if the psf is valid, correctly indexed 
                deconv_value = np.mean(deconv_list_use[psf_idx][photon_idx])
                ax.scatter(mi_value, deconv_value, color=color, marker=marker_for_psf(psf_name), s=50, zorder=100)
                # add to lists to track later 
                mi_means_across_photons.append(mi_value)
                deconv_means_across_photons.append(deconv_value)
                # calculate error bars 
                deconv_lower_across_photons.append(np.percentile(deconv_list_use[psf_idx][photon_idx], 100 - 100 * (1 + confidence_level) / 2))
                deconv_upper_across_photons.append(np.percentile(deconv_list_use[psf_idx][photon_idx], 100 * (1 + confidence_level) / 2))
        mi_means_across_photons = np.array(mi_means_across_photons)
        deconv_means_across_photons = np.array(deconv_means_across_photons)
        ax.plot(mi_means_across_photons, deconv_means_across_photons, '--', color='grey', alpha=1, linewidth=2)
        ax.fill_between(mi_means_across_photons, deconv_lower_across_photons, deconv_upper_across_photons, color='grey', alpha=0.3, linewidth=0, zorder=-100)

ax.set_xlabel('Mutual Information (bits per pixel)')
ax.set_ylabel('Deconvolution Performance, {}'.format(metric_name))
clear_spines(ax)


# legend
# ax.scatter([], [], color='k', marker='x', label='No PSF')
ax.scatter([], [], color='k', marker='o', label='One Lens')
#ax.scatter([], [], color='k', marker='d', label='Two Lens')
#ax.scatter([], [], color='k', marker='v', label='Three Lens')
ax.scatter([], [], color='k', marker='s', label='Four Lens')
#ax.scatter([], [], color='k', marker='p', label='Five Lens')
ax.scatter([], [], color='k', marker='*', label='Diffuser')
#ax.scatter([], [], color='k', marker='P', label='Aperture')

ax.legend(loc='lower right', frameon=True)
ax.set_xlim([0, None])



norm = mpl.colors.Normalize(vmin=min_log_photons, vmax=max_log_photons)
sm = plt.cm.ScalarMappable(cmap=colormap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, ticks=(np.log(valid_photon_counts)))
# set tick labels
cbar.ax.set_yticklabels(valid_photon_counts)


cbar.set_label('Mean Photon Count')


#plt.savefig('{}_vs_MI_with_confidence_intervals_log_photons.pdf'.format(metric_name), bbox_inches='tight', transparent=True)

In [ ]:
estimator_type = 1 # 0 for gaussian, 1 for pixelcnn
metric_type = 0 # 0 for MSE, 1 for PSNR
valid_psfs = [0, 1, 2]
valid_photon_counts = [20, 40, 80, 160, 320]
print([psf_names[i] for i in valid_psfs])

In [ ]:
confidence_level = 0.9
# using min-valued MI estimates 
mi_estimate_lists = [gaussian_mi_estimates_across_psfs, pixelcnn_mi_estimates_across_psfs]
#deconv_estimate_lists = [mse_across_psfs, psnr_across_psfs]
deconv_estimate_lists = [mse_lists_across_psfs, psnr_lists_across_psfs] # use full list versions
metric_names = ['MSE', 'PSNR']
# classifier array is classifier_all_trials_across_psfs, 4x9x10 array. 4 psfs, 9 photon counts, 10 trials on each one 

fig, ax = plt.subplots(1, 1, figsize=(7, 5))

mi_list_use = mi_estimate_lists[estimator_type] # use pixelcnn or gaussian, choose pixelcnn 
deconv_list_use = deconv_estimate_lists[metric_type] # use mse or psnr, choose psnr
metric_name = metric_names[metric_type]

for psf_idx, psf_name in enumerate(psf_names):
    if psf_idx in valid_psfs:
        mi_means_across_photons = [] # track mean MI values to make trendline 
        deconv_means_across_photons = [] # track mean deconvolution values to make trendline
        deconv_lower_across_photons = [] # track lower bounds
        deconv_upper_across_photons = [] # track upper bounds
        for photon_idx, photon_count in enumerate(mean_photon_count_list):
            if photon_count in valid_photon_counts:
                # load mean values and colors to plot 
                color = color_for_photon_level(photon_count)
                mi_value = mi_list_use[psf_idx][photon_idx] # only use an MI value if the psf is valid, correctly indexed 
                deconv_value = np.mean(deconv_list_use[psf_idx][photon_idx])
                ax.scatter(mi_value, deconv_value, color=color, marker=marker_for_psf(psf_name), s=50, zorder=100)
                # add to lists to track later 
                mi_means_across_photons.append(mi_value)
                deconv_means_across_photons.append(deconv_value)
                # calculate error bars 
                deconv_lower_across_photons.append(np.percentile(deconv_list_use[psf_idx][photon_idx], 100 - 100 * (1 + confidence_level) / 2))
                deconv_upper_across_photons.append(np.percentile(deconv_list_use[psf_idx][photon_idx], 100 * (1 + confidence_level) / 2))
        mi_means_across_photons = np.array(mi_means_across_photons)
        deconv_means_across_photons = np.array(deconv_means_across_photons)
        ax.plot(mi_means_across_photons, deconv_means_across_photons, '--', color='grey', alpha=1, linewidth=2)
        ax.fill_between(mi_means_across_photons, deconv_lower_across_photons, deconv_upper_across_photons, color='grey', alpha=0.3, linewidth=0, zorder=-100)

ax.set_xlabel('Mutual Information (bits per pixel)')
ax.set_ylabel('Deconvolution Performance, {}'.format(metric_name))
clear_spines(ax)


# legend
# ax.scatter([], [], color='k', marker='x', label='No PSF')
ax.scatter([], [], color='k', marker='o', label='One Lens')
#ax.scatter([], [], color='k', marker='d', label='Two Lens')
#ax.scatter([], [], color='k', marker='v', label='Three Lens')
ax.scatter([], [], color='k', marker='s', label='Four Lens')
#ax.scatter([], [], color='k', marker='p', label='Five Lens')
ax.scatter([], [], color='k', marker='*', label='Diffuser')
#ax.scatter([], [], color='k', marker='P', label='Aperture')

ax.legend(loc='upper right', frameon=True)
ax.set_xlim([0, None])



norm = mpl.colors.Normalize(vmin=min_log_photons, vmax=max_log_photons)
sm = plt.cm.ScalarMappable(cmap=colormap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, ticks=(np.log(valid_photon_counts)))
# set tick labels
cbar.ax.set_yticklabels(valid_photon_counts)


cbar.set_label('Mean Photon Count')


#plt.savefig('{}_vs_MI_with_confidence_intervals_log_photons.pdf'.format(metric_name), bbox_inches='tight', transparent=True)